In [72]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
"""from keras.models import Sequential
from keras.layers import Dense"""

from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.models import Sequential

import matplotlib.patches as mpatches
from keras.layers import Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score
import tensorflow as tf
from tensorflow.keras.metrics import Accuracy
import keras as K
from keras.layers import Dense,RepeatVector
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils.vis_utils import plot_model
import os

In [66]:
REL_FEATURES = [' _dewptm', ' _fog', ' _hail', ' _hum', ' _pressurem', ' _rain', ' _snow',
                ' _thunder', ' _tornado', ' _vism', ' _wspdm', 'night', 'morning',
                'noon', 'evening', 'month_cos', 'month_sin', 'hour_cos', 'hour_sin', 'week_cos', 'week_sin']
LABEL = ['Temp']

# Time Series Forecast using LSTM

In [67]:
def df_train_test(model_name):
    df_train = pd.read_csv(f'df_{model_name}_train.csv')
    df_test = pd.read_csv(f'df_{model_name}_test.csv')
    return df_train, df_test

In [68]:
def main_func(model_name, rel_features=REL_FEATURES, label=LABEL, timestep=28):
    df_train, df_test = df_train_test(model_name)

    df_train.set_index('datetime', inplace= True)
    df_test.set_index('datetime', inplace= True)

    X_train = df_train[rel_features].values.astype('float32')
    X_test = df_test[rel_features].values.astype('float32')

    Y_train = df_train[label].values.astype('float32')
    Y_test = df_test[label].values.astype('float32')

    for j, df in enumerate([[X_train, Y_train], [X_test, Y_test]]):
        x, y = df
        X = []
        Y = []

        for i in range(len(x)- (timestep)):
            X.append(x[i:i+timestep])
            Y.append(y[i:i+timestep])

        X = np.asanyarray(X)
        Y = np.asanyarray(Y)

        if j == 0:
            X_train = X
            Y_train = Y
        else:
            X_test = X
            Y_test = Y

    #the model
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(timestep, len(REL_FEATURES))))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    #model.compile(loss='mse', optimizer='adam', metrics=[accuracy_per_obs])
    history = model.fit(X_train,Y_train, epochs=100, verbose=0, validation_data=(X_test, Y_test) )

    preds_train = model.predict(X_train)
    preds_test = model.predict(X_test)

    preds_train = np.round(preds_train)
    preds_test = np.round(preds_test)

    mse_train = mean_squared_error(Y_train.ravel(), preds_train.ravel())
    mse_test = mean_squared_error(Y_test.ravel(), preds_test.ravel())

    acc_train = accuracy_score(Y_train.ravel(), preds_train.ravel())
    acc_test = accuracy_score(Y_test.ravel(), preds_test.ravel())

    print(f'MSE train: {mse_train}')
    print(f'MSE test: {mse_test}')

    print(f'Accuracy train: {acc_train}')
    print(f'Accuracy test: {acc_test}')

    """plt.figure(figsize=(25,10))
    plt.plot(Y_test.ravel(), 'blue', linewidth=0.1)
    plt.plot(preds_test.ravel(),'r' , linewidth=0.1)
    plt.legend(('Test','Predicted'))
    plt.show()"""

    try:
        # summarize history for loss
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title(f'model {model_name} loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.savefig(f'{model_name}_loss.png')
        plt.close()
    except:
        pass

    return mse_train, mse_test, acc_train, acc_test

    """# summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()"""

<a id="1"></a> <br>
# **CNN-LSTM Model**

In [ ]:
data = {}

for freq_range in [[str(i) for i in range(1, 13)],  # month
                    ['autumn', 'winter', 'spring', 'monsoon', 'summer'],  # season
                    ['full']]:  # full year
    # all day
    for freq_part in freq_range:
        for day_part in ['all', 'night', 'morning', 'noon', 'evening']:
            model_name = '_'.join([freq_part, day_part])
            print(model_name)
            if day_part != 'all':
                mse_train, mse_test, acc_train, acc_test = main_func(model_name, timestep=7)
            else:
                mse_train, mse_test, acc_train, acc_test = main_func(model_name)
            data[model_name] = [mse_train, mse_test, acc_train, acc_test]

df = pd.DataFrame.from_dict(data, orient='index', columns=['mse_train', 'mse_test', 'acc_train', 'acc_test'])
df.to_csv(r'results.csv', index = True)

#Analyze results

In [74]:
df

,mse_train,mse_test,acc_train,acc_test
1_all,0.083531,0.242560,0.917181,0.757440
1_night,0.202460,0.226190,0.804636,0.773810
1_morning,0.451827,0.613095,0.600854,0.529762
1_noon,0.480476,0.392857,0.575714,0.678571
1_evening,0.261509,0.273810,0.745610,0.726190
...,...,...,...,...
full_all,0.033941,0.269527,0.966838,0.745346
full_night,0.204343,0.241421,0.799317,0.762171
full_morning,0.284979,0.314205,0.728494,0.699037
full_noon,0.201110,0.246753,0.801460,0.760552


In [70]:
def accuracy_per_obs(y_true, y_pred):
    print(np.add(y_true, 0))
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    """y_true = tf.reshape(y_true, [len(y_true)])
    y_pred = tf.reshape(y_pred, [len(y_true)])
    return Accuracy().update_state(y_true, y_pred)"""
    #return accuracy_score(y_true, y_pred)
    return Accuracy(y_true, y_pred)

In [71]:
"""#too complicated
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=2, activation='relu', input_shape=(len(REL_FEATURES),1)))
model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(RepeatVector(30))
model.add(LSTM(128, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(X_train,Y_train,epochs=50, verbose=0 )"""

"#too complicated\nmodel = Sequential()\nmodel.add(Conv1D(filters=256, kernel_size=2, activation='relu', input_shape=(len(REL_FEATURES),1)))\nmodel.add(Conv1D(filters=128, kernel_size=2, activation='relu'))\nmodel.add(MaxPooling1D(pool_size=2))\nmodel.add(Flatten())\nmodel.add(RepeatVector(30))\nmodel.add(LSTM(128, activation='relu'))\nmodel.add(Dense(100, activation='relu'))\nmodel.add(Dense(1))\nmodel.compile(loss='mse', optimizer='adam')\nmodel.fit(X_train,Y_train,epochs=50, verbose=0 )"

In [ ]:
"""from google.colab import files
for freq_range in [[str(i) for i in range(1, 13)],  # month
                    ['autumn', 'winter', 'spring', 'monsoon', 'summer'],  # season
                    ['full']]:  # full year
    # all day
    for freq_part in freq_range:
        for day_part in ['all', 'night', 'morning', 'noon', 'evening']:
            model_name = '_'.join([freq_part, day_part])
            print(model_name)
            f = f'{model_name}_loss.png'
            files.download(f)
files.download('results.csv')"""